In [58]:
import numpy as np
from scipy.stats import norm
from astropy.modeling import models

In [4]:
tab = np.loadtxt('input_ex3.dat',comments="#")

In [86]:
# train our Naive Bayes model using conditional independence
ok_ind = np.where(tab[:,7] == 1)[0]
m1 = np.mean(tab[ok_ind,1:7],axis = 0)
std1 = np.std(tab[ok_ind,1:7],axis = 0,ddof=1)

settler_ind = np.where(tab[:,7] == 2)[0]
m2 = np.mean(tab[settler_ind,1:7],axis = 0)
std2 = np.std(tab[settler_ind,1:7],axis = 0,ddof=1)

solids_ind = np.where(tab[:,7] == 3)[0]
m3 = np.mean(tab[solids_ind,1:7],axis = 0)
std3 = np.std(tab[solids_ind,1:7],axis = 0,ddof=1)

prior_ok = len(ok_ind)/np.float(len(tab[:,7]))
prior_settler = len(settler_ind)/np.float(len(tab[:,7]))
prior_solids = len(solids_ind)/np.float(len(tab[:,7]))

In [87]:
print('mean: ',m1,m2,m3)
print('standard deviation: ',std1,std2,std3)
print(prior_ok,prior_settler,prior_solids)
print(len(ok_ind),len(tab[:,7]))

mean:  [  1.89000000e+02   3.12500000e+00   1.86050000e+03   1.80000000e+01
   5.40000000e-02   2.03600000e+03] [  2.00800000e+02   4.40000000e+00   1.25120000e+03   9.80000000e+01
   1.01800000e+00   1.37200000e+03] [ 1301.      32.5   1621.      49.1   1293.     832.85]
standard deviation:  [  4.54165902e+01   2.50000000e-01   3.71402298e+02   6.05530071e+00
   9.74029431e-02   5.32191068e+02] [  55.12893977    1.78185297  116.24413964   23.37733946    1.52663683
  142.5780488 ] [ 485.44000659   11.95826074  453.03789393   37.75588254  430.95088661
  958.31223339]
0.3076923076923077 0.38461538461538464 0.3076923076923077
4 13


In [88]:
ok_model = models.Gaussian1D(mean=m1[0],stddev=std1[0],amplitude=1.0/(np.sqrt(2.0*np.pi)*std1[0])) 
settler_model = models.Gaussian1D(mean=m2[0],stddev=std2[0],amplitude=1.0/(np.sqrt(2.0*np.pi)*std2[0])) 
solid_model = models.Gaussian1D(mean=m3[0],stddev=std3[0],amplitude=1.0/(np.sqrt(2.0*np.pi)*std3[0])) 

for i in np.arange(1,len(m1)):
    ok_model = ok_model & models.Gaussian1D(mean=m1[i],stddev=std1[i],amplitude=1.0/(np.sqrt(2.0*np.pi)*std1[i]))
    settler_model = settler_model & models.Gaussian1D(mean=m2[i],stddev=std2[i],amplitude=1.0/(np.sqrt(2.0*np.pi)*std2[i])) 
    solid_model = solid_model & models.Gaussian1D(mean=m3[i],stddev=std3[i],amplitude=1.0/(np.sqrt(2.0*np.pi)*std3[i]))

In [91]:
ok_model.__class__
print(settler_model.__class__)
print(settler_model.mean_0)

print()

<class '__main__.CompoundModel37'>
Name: CompoundModel37
Inputs: ('x000', 'x100', 'x00', 'x10', 'x0', 'x1')
Outputs: ('y000', 'y100', 'y00', 'y10', 'y0', 'y1')
Fittable parameters: ('amplitude_0', 'mean_0', 'stddev_0', 'amplitude_1', 'mean_1', 'stddev_1', 'amplitude_2', 'mean_2', 'stddev_2', 'amplitude_3', 'mean_3', 'stddev_3', 'amplitude_4', 'mean_4', 'stddev_4', 'amplitude_5', 'mean_5', 'stddev_5')
Expression: [0] & [1] & [2] & [3] & [4] & [5]
Components: 
    [0]: <Gaussian1D(amplitude=0.007236530977679192, mean=200.8, stddev=55.128939768509966)>

    [1]: <Gaussian1D(amplitude=0.22389180675340853, mean=4.4, stddev=1.7818529681205462)>

    [2]: <Gaussian1D(amplitude=0.0034319345615738826, mean=1251.2, stddev=116.24413963723075)>

    [3]: <Gaussian1D(amplitude=0.01706534146741944, mean=98.0, stddev=23.37733945512192)>

    [4]: <Gaussian1D(amplitude=0.26132101196476487, mean=1.018, stddev=1.5266368264914876)>

    [5]: <Gaussian1D(amplitude=0.002798062420935983, mean=1372.0, stddev

In [92]:
input_data = (222.0,4.5,1518.0,74.0,0.25,1642.0)
ok_prob = np.product(ok_model(*input_data))*prior_ok
settler_prob = np.product(settler_model(*input_data))*prior_settler
solid_prob = np.product(solid_model(*input_data))*prior_solids
normalization = ok_prob + settler_prob + solid_prob
print(ok_model(*input_data))
print(ok_prob,settler_prob,solid_prob)
print(ok_prob/normalization,settler_prob/normalization,solid_prob/normalization)

(0.006746073391719539, 4.3079040170173104e-07, 0.0007021012639342934, 1.7650745447473623e-20, 0.540838872464162, 0.000569934270909125)
3.41577453757e-36 1.53837178939e-13 1.00668352227e-21
2.22038296514e-23 0.999999993456 6.54382459835e-09
